In [14]:
import sys
sys.path.append("../")

In [2]:
from Data.BiGCN_Dataloader import *

In [3]:
!pwd

In [4]:
os.environ['DaMSTF_PATH'] = '/data01/home/scy0798/run/MLLu/da-mstf_-paper_-codes'

In [5]:
dir_name = "/data01/home/scy0798/run/MLLu/pheme-rnr-dataset/"

In [6]:
e_l = [fname for fname in [os.path.join(dir_name, path) for path in os.listdir(dir_name)] if os.path.isdir(fname)]

In [7]:
rst = load_data(e_l[:4], e_l[4:])

In [8]:
from BaseModel.BiGCN import *

In [9]:
    Tf_Idf_twitter_file = f"{os.environ['DaMSTF_PATH']}/Caches/TfIdf_twitter.pkl"
    if os.path.exists(Tf_Idf_twitter_file):
        with open(Tf_Idf_twitter_file, "rb") as fr:
            tv = pickle.load(fr)

In [10]:
    lvec = TFIDFBasedVecV2(tv, 20,
                         embedding_size=100,
                         w2v_dir="../../glove_en/",
                         emb_update=True,
                         grad_preserve=True)

In [11]:
lvec.embedding.weight

In [12]:
def obtain_BiGCN(pretrained_vectorizer, device=None):
    if device is None:
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
    lvec = TFIDFBasedVecV2(pretrained_vectorizer, 20,
                         embedding_size=100,
                         w2v_dir="../../glove_en/",
                         emb_update=True,
                         grad_preserve=True).to(device)
    prop = BiGCNV2(100, 256).to(device)
    cls = nn.Linear(1024, 2).to(device)
    BiGCN_model = TwitterBiGCN(lvec, prop, cls)
    return BiGCN_model

In [13]:
model = obtain_BiGCN(tv)

In [15]:
class BiGCNTrainer(RumorBaseTrainer):
    def __init__(self, log_dir, tokenizer:TfidfVectorizer):
        super(BiGCNTrainer, self).__init__()
        self.running_dir = log_dir
        if not os.path.exists(self.running_dir):
            os.system(f"mkdir {self.running_dir}")
        self.tokenizer = tokenizer
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    def collate_fn(self, items):
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
        tfidf_arr = torch.cat(
            [item[0] for item in items],
            dim=0
        )
        TD_graphs = [item[1] for item in items]
        BU_graphs = [item[2] for item in items]
        labels = [item[3] for item in items]
        topic_labels = [item[4] for item in items]
        num_nodes = [g.num_nodes() for g in TD_graphs]
        big_g_TD = dgl.batch(TD_graphs)
        big_g_BU = dgl.batch(BU_graphs)
        A_TD = big_g_TD.adjacency_matrix().to_dense().to(device)
        A_BU = big_g_BU.adjacency_matrix().to_dense().to(device)
        return tfidf_arr, num_nodes, A_TD, A_BU, \
               torch.tensor(labels), torch.tensor(topic_labels)

    def dataset2dataloader(self, dataset:FastBiGCNDataset, shuffle=False, batch_size=32):
        if shuffle:
            idxs = random.sample(range(len(dataset)), len(dataset))*2
        else:
            idxs = [*range(len(dataset))]*2
        @background(max_prefetch=5)
        def dataloader():
            for start in range(0, len(dataset), batch_size):
                batch_idxs = idxs[start:start+batch_size]
                items = [dataset[index] for index in batch_idxs]
                yield self.collate_fn(items)
        return dataloader()